In [1]:
# TODO from insights by prof
# only match by labels (no other matching needed, no properties, superclasses, etc.)
# just save labels in list with key is class uri and value is label: if memory is not a problem save in to structures: key value of label and value is class uri
# 5 - 10 min for human, mouse ontologies is reasonable

# Output: in rdf format (see reference_anatomy) just put final matches in there above the threshold (defined by the user). As relation use owl:equivalentClass

In [2]:
#@TODO
# implement additional user inputs: thresholds, weighted average between methods, choose LLM etc.
# implement TF-IDF and cosine similarity

In [3]:
# Required libraries
!pip install sentence_transformers
!pip install pandas
!pip install rdflib
!pip install python-Levenshtein
!pip install scikit-learn
!pip install torch


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


# Project Description
The goal of the project is to develop a simple yet effective ontology alignment framework in Python that focuses on lexical similarity matching. The framework will utilize both string matching techniques and the semantic capabilities of large language models to identify potential alignments between entities (such as classes) in two different ontologies.

### Objectives
1. **Develop an ontology alignment framework** that can process and compare ontologies based on textual content.
2. **Implement lexical similarity matching** using both basic string matching techniques and advanced semantic analysis with embeddings from LLMs.
3. **Output alignments with confidence scores**, enabling users to understand and evaluate the quality and reliability of the suggested alignments.

### Steps to Perform

#### Step 1: Ontology Parsing
- **Goal**: Load and parse the ontologies to be aligned.
- **Tasks**:
  - Utilize libraries like `rdflib` or `owlready2` to read ontology files.
  - Extract relevant textual information (e.g., class names, labels, descriptions).

#### Step 2: Lexical Similarity Matching
This step is divided into two sub-steps: string matching and embeddings matching.

##### a. String Matching
- **Goal**: Implement direct and fuzzy string comparison techniques to find matches based on textual similarity.
- **Tasks**:
  - Perform normalization (e.g., lowercasing, removing special characters).
  - Use string comparison methods (exact match, substring search, edit distance).

##### b. Embeddings Matching Using LLMs
- **Goal**: Use the semantic context provided by LLMs to match terms based on their meanings.
- **Tasks**:
  - Generate embeddings for the textual content of each ontology using models from the Hugging Face Transformers library.
  - Calculate similarity scores between embeddings (e.g., using cosine similarity).

#### Step 3: Combining and Filtering
- **Goal**: Aggregate results from both matching techniques and refine the output.
- **Tasks**:
  - Combine scores from string and embeddings matching.
  - Apply thresholds to filter out matches with low confidence.
  - Optionally, use simple structural checks to add confidence to matches (e.g., matched entities have similar parent classes).

#### Step 4: Output and Evaluation
- **Goal**: Output the alignment results and provide means for evaluation.
- **Tasks**:
  - Format the output in a structured way (e.g., JSON, CSV) that lists entity pairs and their matching scores.
  - If possible, evaluate the effectiveness using known benchmarks or test cases to calculate precision, recall, and F1-score.

### Summary
The project is centered on creating a practical tool for ontology matching, focusing on textual content using both conventional and advanced NLP techniques. By combining string-based and semantic-based approaches, the framework aims to provide robust alignments that are supported by both literal and contextual text similarities. This dual approach enhances the capability of the alignment process, making it more flexible and potentially more accurate than using only one method.

In [4]:
# imports
import rdflib
import pandas as pd
from collections import OrderedDict

**rdflib vs owlready2:**

Interchangeability: Given that OWL is an application of RDF, tools that can parse RDF/XML can generally handle .owl files, and vice versa, provided that the ontology-specific constructs are understood by the tool. This is why libraries like rdflib, which are capable of parsing RDF, are suitable for handling OWL files serialized in RDF/XML format.

Flexibility: Choosing to work with rdflib for general RDF handling and owlready2 for specific ontology manipulations where needed is a flexible approach. It allows you to leverage the strengths of both libraries—rdflib for its robust RDF manipulation and SPARQL querying capabilities, and owlready2 for its ontology-specific features like reasoning and direct manipulation of classes and properties.

# Load/ Parse ontologies

In [5]:
# input paths
onto1_path_in = "test_ontologies/mouse.owl"
onto2_path_in = "test_ontologies/human.owl"


In [6]:
def load_ontology(file_path):
    """
    Loads an ontology from a given file path, which can be in RDF (.rdf) or OWL (.owl) format.

    Args:
    file_path (str): The file path to the ontology file.

    Returns:
    rdflib.Graph: A graph containing the ontology data.
    """
    # Create a new RDF graph
    graph = rdflib.Graph()

    # Bind some common namespaces to the graph
    namespaces = {
        "rdf": rdflib.namespace.RDF,
        "rdfs": rdflib.namespace.RDFS,
        "owl": rdflib.namespace.OWL,
        "xsd": rdflib.namespace.XSD
    }
    for prefix, namespace in namespaces.items():
        graph.namespace_manager.bind(prefix, namespace)

    # Attempt to parse the file
    try:
        graph.parse(file_path, format=rdflib.util.guess_format(file_path))
        print(f"Successfully loaded ontology from {file_path}")
    except Exception as e:
        print(f"Failed to load ontology from {file_path}: {e}")
        return None

    return graph

In [7]:
# load ontologies
onto1_graph = load_ontology(onto1_path_in)
onto2_graph = load_ontology(onto2_path_in)
print(onto1_graph, onto2_graph)

Successfully loaded ontology from test_ontologies/mouse.owl
Successfully loaded ontology from test_ontologies/human.owl
[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']]. [a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].


In [8]:
def preprocess_label(label):
    return str(label).replace("_", " ").strip(" ,.").lower()

### New approach without json and instead dicts

In [9]:
def extract_ontology_details_to_dict(graph):
    # Query for classes
    class_query = """
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?class ?label ?label_dt ?label_lang
    WHERE {
        ?class rdf:type owl:Class.
        OPTIONAL { ?class rdfs:label ?label. BIND(datatype(?label) AS ?label_dt) BIND(lang(?label) AS ?label_lang) }
    }
    """
    classes = graph.query(class_query)
    ontology_labels_dict = OrderedDict()
    labels_list = []

    # Process class results
    for row in classes:
        class_uri, label, label_dt, label_lang = row
        class_key = str(class_uri)
        label_str = preprocess_label(label)

        if label_str is None or label_str == "none":
            continue

        if label_str not in ontology_labels_dict:
            ontology_labels_dict[label_str] = class_key
            labels_list.append(label_str)

    return ontology_labels_dict, labels_list

In [10]:
onto1_dict, onto1_list = extract_ontology_details_to_dict(onto1_graph)
onto2_dict, onto2_list = extract_ontology_details_to_dict(onto2_graph)

In [11]:
onto1_dict

OrderedDict([('mouse anatomy', 'http://mouse.owl#MA_0000001'),
             ('spinal cord grey matter', 'http://mouse.owl#MA_0000002'),
             ('organ system', 'http://mouse.owl#MA_0000003'),
             ('trunk', 'http://mouse.owl#MA_0000004'),
             ('body cavity/lining', 'http://mouse.owl#MA_0000005'),
             ('head/neck', 'http://mouse.owl#MA_0000006'),
             ('limb', 'http://mouse.owl#MA_0000007'),
             ('tail', 'http://mouse.owl#MA_0000008'),
             ('adipose tissue', 'http://mouse.owl#MA_0000009'),
             ('cardiovascular system', 'http://mouse.owl#MA_0000010'),
             ('connective tissue', 'http://mouse.owl#MA_0000011'),
             ('endocrine system', 'http://mouse.owl#MA_0000012'),
             ('hemolymphoid system', 'http://mouse.owl#MA_0000013'),
             ('integumental system', 'http://mouse.owl#MA_0000014'),
             ('muscle', 'http://mouse.owl#MA_0000015'),
             ('nervous system', 'http://mouse.owl#

In [12]:
onto1_list

['mouse anatomy',
 'spinal cord grey matter',
 'organ system',
 'trunk',
 'body cavity/lining',
 'head/neck',
 'limb',
 'tail',
 'adipose tissue',
 'cardiovascular system',
 'connective tissue',
 'endocrine system',
 'hemolymphoid system',
 'integumental system',
 'muscle',
 'nervous system',
 'sensory organ',
 'skeletal system',
 'visceral organ system',
 'back',
 'abdomen/pelvis/perineum',
 'thorax',
 'head',
 'neck',
 'forelimb',
 'hindlimb',
 'lower back',
 'upper back',
 'abdomen',
 'pelvis',
 'chest',
 'thoracic cavity',
 'arm',
 'lower arm',
 'upper arm',
 'elbow',
 'hand',
 'shoulder',
 'wrist',
 'carpus',
 'hand digit',
 'metacarpus',
 'ankle',
 'foot',
 'hip',
 'knee',
 'leg',
 'foot digit',
 'metatarsus',
 'tarsus',
 'lower leg',
 'upper leg',
 'pericardial cavity',
 'peritoneal cavity',
 'pleural cavity',
 'fat',
 'brown fat',
 'white fat',
 'blood',
 'blood vessel',
 'arterial blood vessel',
 'aorta',
 'arteriole',
 'artery',
 'capillary',
 'venous blood vessel',
 'vein',


## 3. Matching

### 3.1. String Matching

In [13]:
def exact_string_match(onto1_dict, onto1_list, onto2_dict, onto2_list):
    exact_matches = {}
    matched_labels1 = set()
    matched_labels2 = set()

    for label1 in onto1_list:
        for label2 in onto2_list:
            if label1 == label2:
                # Creating the formatted match entry
                class1 = onto1_dict[label1]
                class2 = onto2_dict[label2]
                exact_matches[class1] = [label1, class2, label2, 1]
                
                # Tracking matched labels for later removal
                matched_labels1.add(label1)
                matched_labels2.add(label2)
                
    # Remove matched labels from lists and dictionaries
    for label in matched_labels1:
        onto1_list.remove(label)
        del onto1_dict[label]

    for label in matched_labels2:
        onto2_list.remove(label)
        del onto2_dict[label]

    return exact_matches, onto1_dict, onto1_list, onto2_dict, onto2_list


In [14]:
exact_matches, onto1_dict_after_exact, onto1_list_after_exact, onto2_dict_after_exact, onto2_list_after_exact = exact_string_match(onto1_dict, onto1_list, onto2_dict, onto2_list)
print(len(exact_matches))
exact_matches

933


{'http://mouse.owl#MA_0000003': ['organ system',
  'http://human.owl#NCI_C12919',
  'organ system',
  1],
 'http://mouse.owl#MA_0000004': ['trunk',
  'http://human.owl#NCI_C33816',
  'trunk',
  1],
 'http://mouse.owl#MA_0000007': ['limb',
  'http://human.owl#NCI_C12429',
  'limb',
  1],
 'http://mouse.owl#MA_0000009': ['adipose tissue',
  'http://human.owl#NCI_C12472',
  'adipose tissue',
  1],
 'http://mouse.owl#MA_0000010': ['cardiovascular system',
  'http://human.owl#NCI_C12686',
  'cardiovascular system',
  1],
 'http://mouse.owl#MA_0000011': ['connective tissue',
  'http://human.owl#NCI_C12374',
  'connective tissue',
  1],
 'http://mouse.owl#MA_0000012': ['endocrine system',
  'http://human.owl#NCI_C12705',
  'endocrine system',
  1],
 'http://mouse.owl#MA_0000015': ['muscle',
  'http://human.owl#NCI_C13056',
  'muscle',
  1],
 'http://mouse.owl#MA_0000016': ['nervous system',
  'http://human.owl#NCI_C12755',
  'nervous system',
  1],
 'http://mouse.owl#MA_0000018': ['skeletal s

For String Matching we will implement 4 different methods that the user then can chose via a parameter when calling the method.

The metrics we will use are:
- Levenshtein distance
- Jaccard Similarity
- Cosine Similarity
- TF-IDF

In [15]:
import Levenshtein
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def levenshtein_similarity(str1, str2):
    levenshtein_distance = Levenshtein.distance(str1, str2)
    max_length = max(len(str1), len(str2))
    if max_length == 0:
        return 1.0  # Both strings are empty
    normalized_distance = levenshtein_distance / max_length
    similarity = 1 - normalized_distance
    return similarity

def calc_cosine_similarity(str1, str2):
    vectorizer = CountVectorizer()
    count_matrix = vectorizer.fit_transform([str1, str2])
    return cosine_similarity(count_matrix)[0][1]

def jaccard_similarity(str1, str2):
    # Tokenize the strings into sets of words
    set1 = set(str1.split())
    set2 = set(str2.split())

    # Find the intersection and union of the two sets
    intersection = set1.intersection(set2)
    union = set1.union(set2)

    # Calculate the Jaccard score
    if not union:  # Handle the edge case where both strings might be empty
        return 0.0
    return len(intersection) / len(union)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def cosine_vectorize_labels(labels):
    """
    Converts a list of labels into TF-IDF vectors using TfidfVectorizer.

    Args:
    labels (list): List of all labels from both ontologies.

    Returns:
    TfidfVectorizer, scipy.sparse.csr.csr_matrix: The vectorizer and the TF-IDF matrix.
    """
    vectorizer = CountVectorizer()
    count_matrix = vectorizer.fit_transform(labels)
    return vectorizer, count_matrix

def precompute_cosine_similarities(count_matrix):
    """
    Computes the cosine similarity matrix for all pairs of labels in the count matrix.

    Args:
    count_matrix (scipy.sparse.csr.csr_matrix): The matrix containing the count vectors.

    Returns:
    numpy.ndarray: A 2D array of cosine similarity scores.
    """
    return cosine_similarity(count_matrix)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def tfidf_vectorize_labels(labels):
    """
    Converts a list of labels into TF-IDF vectors using TfidfVectorizer.

    Args:
    labels (list): List of all labels from both ontologies.

    Returns:
    TfidfVectorizer, scipy.sparse.csr.csr_matrix: The vectorizer and the TF-IDF matrix.
    """
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(labels)
    return vectorizer, tfidf_matrix


In [18]:
def execute_string_matching(metric, data1, data2):
    """
    Executes the selected matching metric on the provided data.

    Args:
    metric (str): A single letter representing the metric to use.
                  'Levenshtein' for Levenshtein Distance,
                  'Jaccard' for Jaccard Similarity,
                  'Cosine' for Cosine Similarity,
                  'TF-IDF' for TF-IDF Cosine Similarity. 
    data1, data2 (str): The data strings to compare.

    Returns:
    result: The result of the chosen metric computation.
    """
    if metric == 'Levenshtein':
        return levenshtein_similarity(data1, data2)
    elif metric == 'Jaccard':
        return jaccard_similarity(data1, data2)
    else:
        raise ValueError("Invalid metric selection")

In [19]:
def match_ontologies(onto1_dict, onto1_list, onto2_dict, onto2_list, metric, bidirectional=False):
    labels_already_tested_labels = {} # dict to store when labels (of ontology 2) were already tested for label (of ontology 1) => necessary to avoid infite loop
    class_results = {}
    
    for label in onto1_list:
        labels_already_tested_labels[label] = []

    onto2_used_classes = {}

    # create dictionary of index and label for ontology 1 - only needed for cosine and tf-idf
    index_dict_label1 = {}
    for index, label1 in enumerate(onto1_list):
        index_dict_label1[label1] = index

    # here: call vectorization functions
    if metric == "Cosine":
        all_labels = onto1_list + onto2_list
        vectorizer, matrix = cosine_vectorize_labels(all_labels)
        similarity_matrix = precompute_cosine_similarities(matrix)
    elif metric == "TF-IDF":
        all_labels = onto1_list + onto2_list
        vectorizer, matrix = tfidf_vectorize_labels(all_labels)
        similarity_matrix = precompute_cosine_similarities(matrix)

    while onto1_list: # loop over labels of ontology 1 until empty
        #print(len(onto1_list))
        label1 = onto1_list.pop() # remove the last element in the list => removing the last (instead of first) makes things easier and less error prone
        # labels that got added again cause a better match was found (see later step) will be appended to the end and therefore handled immediately

        # Match from Ontology 1 to Ontology 2
        label_result = [label1, "", "", 0]
        best_score = 0
        already_tested_labels = set(labels_already_tested_labels[label1])
        for index2, label2 in enumerate(onto2_list):
            if label2 not in already_tested_labels: # check that label wasn't already checked in previous run
                if metric == "Levenshtein" or metric == "Jaccard":
                    matching_score = execute_string_matching(metric, label1, label2) # calculate string matching score
                elif metric == "Cosine" or metric == "TF-IDF":
                    matching_score = similarity_matrix[index_dict_label1[label1]][len(index_dict_label1) + index2]
                
                # If a perfect match is found, stop iterating over labels for this entry
                if matching_score == 1: # handle perfect match
                    best_score = matching_score
                    label_result = [label1, "", label2, best_score]
                    break # stop searching for matches cause perfect match found
                # Check if a match for this label has been found before
                if matching_score > best_score: # handle higher score then before
                    label_result[2] = label2
                    best_score = matching_score

        label_result[3] = best_score # save best score in label_result
        label_with_best_score = label_result[2] # get label that achieved the best score

        class_uri = onto1_dict[label1] # get the class_uri of the currently checked label in ontology 1
        if label_result[3] == 0 and label_with_best_score == '': # handle if no match was found
            class_results[class_uri] = label_result
        else:
            class2_uri = onto2_dict[label_with_best_score] # get class_uri of the label with best match
            label_result[1] = class2_uri # save class_uri instead of label => TODO maybe change to not manipulate label_result as it is confusing for later steps

            if class2_uri not in onto2_used_classes: # check if class found of ontology 2 is NOT already used by other class in ontology 1
                if class_uri not in class_results: # handle no entry exists yet for that class
                    class_results[class_uri] = label_result
                    onto2_used_classes[class2_uri] = class_uri
                    labels_already_tested_labels[label1].append(label_with_best_score)
                elif label_result[3] > class_results[class_uri][3]: # handle entry exist but now higher score was found with another label of the class (handles multiple labels)
                        class_results[class_uri] = label_result
                        onto2_used_classes[class2_uri] = class_uri
                        del onto2_used_classes[class_results[class_uri][1]] # delete old class2 as it is now free to match again with other labels
                        labels_already_tested_labels[label1].append(label_with_best_score)
            else: # class of ontology 2 already in use
                result_current_class_in_use = onto2_used_classes[class2_uri] # get class uri of class that uses that class of ontology 2
                if label_result[3] > class_results[result_current_class_in_use][3]: # if score of the new found match is higher than the current assigned one
                    class_results[class_uri] = label_result # set the class of ontology 2 to that current class
                    onto2_used_classes[class2_uri] = class_uri # overwrite the use of that class to new class of ontology 1
                    old_used_label = class_results[result_current_class_in_use][0]
                    labels_already_tested_labels[old_used_label].append(label_with_best_score)
                    onto1_list.append(old_used_label) # add the old used label again to list again that gets iterated as it now doesnt have a match anymore
                    class_results[result_current_class_in_use] = ["", "", "", 0] # set result of earlier class to None (could also be remmoved but that way later we can handle if no match found)
                else: # handle not a higher score
                    labels_already_tested_labels[label1].append(label_with_best_score) # add the label to the already_tested_labels
                    onto1_list.append(label1) # append currently check label again as it needs to handled again with the new information of already_tested_labels

    # TODO implement bidirectional matching

    # OLD_TO-DO important: currently it takes the best match found for the current class of the ontology 1.
    # But it doesnt take into account if a later class has a higher score with that class and therefore would be better suited
    # Solution: make dict and always take one element that gets removed. If later another element matches with the class matched with
    # the previous element the earlier removed element get added again and the value of that elements gets assigned to the higher value element

    return class_results

In [20]:
string_matching_results = match_ontologies(onto1_dict_after_exact, onto1_list_after_exact, onto2_dict_after_exact, onto2_list_after_exact, 'Cosine')

In [21]:
string_matching_results

{'http://mouse.owl#MA_0002758': ['levator auris longus',
  'http://human.owl#NCI_C32704',
  'gyrus longus insulae',
  0.3333333333333334],
 'http://mouse.owl#MA_0002757': ['periolivary nucleus',
  'http://human.owl#NCI_C32337',
  'cochlear nucleus',
  0.4999999999999999],
 'http://mouse.owl#MA_0002756': ['lateral spinal nucleus',
  'http://human.owl#NCI_C32951',
  'lateral vestibular nucleus',
  0.6666666666666669],
 'http://mouse.owl#MA_0002755': ['lateral cervical nucleus',
  'http://human.owl#NCI_C32944',
  'lateral posterior nucleus of the thalamus',
  0.4714045207910318],
 'http://mouse.owl#MA_0002754': ['neocortex', '', '', 0],
 'http://mouse.owl#MA_0002753': ['spinal cord intermediolateral nucleus',
  'http://human.owl#NCI_C12894',
  'thoracic spinal cord',
  0.5773502691896258],
 'http://mouse.owl#MA_0002752': ['nucleus approprius',
  'http://human.owl#NCI_C33861',
  'ventroanterior nucleus of the thalamus',
  0.3162277660168379],
 'http://mouse.owl#MA_0002751': ['spinal cord v

In [22]:
print(len(onto1_dict))
print(len(string_matching_results))
# TODO one element is always missing: length of classes in onto_1 is 2739 but resulting matches is 2738. Same by LLM

1805
1804


In [23]:
string_matching_results

{'http://mouse.owl#MA_0002758': ['levator auris longus',
  'http://human.owl#NCI_C32704',
  'gyrus longus insulae',
  0.3333333333333334],
 'http://mouse.owl#MA_0002757': ['periolivary nucleus',
  'http://human.owl#NCI_C32337',
  'cochlear nucleus',
  0.4999999999999999],
 'http://mouse.owl#MA_0002756': ['lateral spinal nucleus',
  'http://human.owl#NCI_C32951',
  'lateral vestibular nucleus',
  0.6666666666666669],
 'http://mouse.owl#MA_0002755': ['lateral cervical nucleus',
  'http://human.owl#NCI_C32944',
  'lateral posterior nucleus of the thalamus',
  0.4714045207910318],
 'http://mouse.owl#MA_0002754': ['neocortex', '', '', 0],
 'http://mouse.owl#MA_0002753': ['spinal cord intermediolateral nucleus',
  'http://human.owl#NCI_C12894',
  'thoracic spinal cord',
  0.5773502691896258],
 'http://mouse.owl#MA_0002752': ['nucleus approprius',
  'http://human.owl#NCI_C33861',
  'ventroanterior nucleus of the thalamus',
  0.3162277660168379],
 'http://mouse.owl#MA_0002751': ['spinal cord v

In [24]:
#import itertools
import time
import torch
import logging
from sentence_transformers import SentenceTransformer, util

def calculate_label_similarity_llm(model_name, onto1_dict, onto2_dict):
    """
    Calculate cosine similarity between pairs of labels from two sets and return the results in a dictionary.
    Each key in the dictionary is the class URI from ontology 1, and each value is a list of tuples,
    each containing the label from ontology 2, the class URI from ontology 2, and the similarity score.

    Parameters:
    model_name (str): Name of the Sentence Transformer model to be used.
    onto1_dict (OrderedDict): Dictionary where keys are labels and values are class URIs for the first ontology.
    onto2_dict (OrderedDict): Dictionary where keys are labels and values are class URIs for the second ontology.

    Returns:
    dict: A dictionary with class URIs from the first ontology as keys and lists of tuples (label, class URI, score) from the second ontology as values.
    """
    
    # Initialize logging
    logging.basicConfig(level=logging.ERROR)
    
    # Determine if a GPU is available and set the device accordingly
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    try:
        model = SentenceTransformer(model_name, device=device)
        print(f"Using model: {model_name} on device: {device}")
    except Exception as e:
        logging.error(f"Error loading model: {e}")
        # Fallback to a default model if there's an error
        fallback_model_name = 'all-MiniLM-L6-v2'
        model = SentenceTransformer(fallback_model_name, device=device)
        print(f"Using fallback model: {fallback_model_name} on device: {device}")
    
    onto1_labels, onto1_classes = zip(*onto1_dict.items())
    onto2_labels, onto2_classes = zip(*onto2_dict.items())

    onto1_label_embeddings = model.encode(list(onto1_labels), convert_to_tensor=True, device=device)
    onto2_label_embeddings = model.encode(list(onto2_labels), convert_to_tensor=True, device=device)

    similarity_scores = util.pytorch_cos_sim(onto1_label_embeddings, onto2_label_embeddings)

    # Initialize the dictionary to hold results
    results_dict = {}

    # Fill the dictionary with similarity scores
    for i, onto1_class in enumerate(onto1_classes):
        results_dict[onto1_class] = {}
        for j, onto2_class in enumerate(onto2_classes):
            results_dict[onto1_class][onto2_class] = similarity_scores[i][j].item()

    # Sort the dictionary entries by similarity score within each onto1_class
    sorted_results_dict = {}
    for onto1_class in results_dict:
        sorted_onto2_classes = sorted(results_dict[onto1_class].items(), key=lambda x: x[1], reverse=True)
        sorted_results_dict[onto1_class] = dict(sorted_onto2_classes)

    return sorted_results_dict

# Record start time
start_time = time.time()

dict_similarity_scores_llm = calculate_label_similarity_llm("all-MiniLM-L12-v2", onto1_dict_after_exact, onto2_dict_after_exact)

# Record end time
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

print(f"Time taken: {elapsed_time} seconds")

/Users/tildietrich/.pyenv/versions/3.10.12/envs/kg_ontology_matching_3.10.12/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Using model: all-MiniLM-L12-v2 on device: cpu
Time taken: 18.259869813919067 seconds


In [25]:
# Print the first entries from the dictionary
for onto1_class in list(dict_similarity_scores_llm.keys())[:20]:
    print(f"{onto1_class}: {dict_similarity_scores_llm[onto1_class]}")

http://mouse.owl#MA_0000001: {'http://human.owl#NCI_C28460': 0.5620852708816528, 'http://human.owl#NCI_C12934': 0.5187564492225647, 'http://human.owl#NCI_C12958': 0.5052818059921265, 'http://human.owl#NCI_C33218': 0.47970739006996155, 'http://human.owl#NCI_C12646': 0.473985493183136, 'http://human.owl#NCI_C13052': 0.4669102728366852, 'http://human.owl#NCI_C52701': 0.4637256860733032, 'http://human.owl#NCI_C32977': 0.45982518792152405, 'http://human.owl#NCI_C52703': 0.4525844156742096, 'http://human.owl#NCI_C12433': 0.4464266002178192, 'http://human.owl#NCI_C41409': 0.4459545314311981, 'http://human.owl#NCI_C12644': 0.4436716139316559, 'http://human.owl#NCI_C13111': 0.4416525065898895, 'http://human.owl#NCI_C32472': 0.44156646728515625, 'http://human.owl#NCI_C32475': 0.43452373147010803, 'http://human.owl#NCI_C33147': 0.43324828147888184, 'http://human.owl#NCI_C32149': 0.42940786480903625, 'http://human.owl#NCI_C32861': 0.42734840512275696, 'http://human.owl#NCI_C33778': 0.4251500368118

In [26]:
def set_new_match(class1_uri, class2_uri, score, onto2_used_classes, class_results):
    """Set a new match for class1_uri and class2_uri."""
    onto2_used_classes[class2_uri] = class1_uri
    class_results[class1_uri] = ["", class2_uri, "", score]  # label is empty for now

def update_matching(new_class1_uri, class2_uri, new_score, old_class1_uri, onto2_used_classes, class_results, onto1_class_list):
    """Update matches when a better score is found."""
    onto2_used_classes[class2_uri] = new_class1_uri
    class_results[old_class1_uri] = ["", "", "", 0]  # Clear old match
    class_results[new_class1_uri] = ["", class2_uri, "", new_score]
    onto1_class_list.append(old_class1_uri)

def reevaluate(class1_uri, class2_uri, already_tested_classes, onto1_class_list):
    """Re-add class1_uri for re-evaluation."""
    onto1_class_list.append(class1_uri)
    already_tested_classes[class1_uri].add(class2_uri)

def perform_matching_llm(dict_similarity_scores_llm):
    # Initialize dictionaries and lists
    already_tested_classes = {}
    class_results = {}
    onto2_used_classes = {}
    onto1_class_list = list(dict_similarity_scores_llm.keys())

    while onto1_class_list:
        class1_uri = onto1_class_list.pop()
        already_tested_classes[class1_uri] = already_tested_classes.get(class1_uri, set())

        # Iterate over each class2_uri and score from the pre-sorted dictionary
        for class2_uri, score in dict_similarity_scores_llm[class1_uri].items():
            if class2_uri not in already_tested_classes[class1_uri]:
                already_tested_classes[class1_uri].add(class2_uri)  # Mark this class2_uri as tested

                if score >= 0.99:  # Check for a perfect match
                    set_new_match(class1_uri, class2_uri, score, onto2_used_classes, class_results)
                    break  # Found a perfect match, skip further checks for this class1_uri

                # If no perfect match, check if it's not already linked
                if class2_uri not in onto2_used_classes:
                    set_new_match(class1_uri, class2_uri, score, onto2_used_classes, class_results)
                    break  # Successfully linked, no need to continue

                # If already linked, check if the new score is better
                elif score > class_results[onto2_used_classes[class2_uri]][3]:
                    old_class1_uri = onto2_used_classes[class2_uri]
                    update_matching(class1_uri, class2_uri, score, old_class1_uri, onto2_used_classes, class_results, onto1_class_list)
                    break  # Updated the link, no need to continue
            else:
                # This class2_uri was already checked, continue to the next
                continue

    return class_results

# Call function
matched_results_llm = perform_matching_llm(dict_similarity_scores_llm)

In [27]:
# Print the first entries from the dictionary
for onto1_class in list(matched_results_llm.keys())[:20]:
    print(f"{onto1_class}: {matched_results_llm[onto1_class]}")

http://mouse.owl#MA_0002758: ['', 'http://human.owl#NCI_C12315', '', 0.4048202633857727]
http://mouse.owl#MA_0002757: ['', 'http://human.owl#NCI_C41416', '', 0.6656939387321472]
http://mouse.owl#MA_0002756: ['', 'http://human.owl#NCI_C12969', '', 0.7062833905220032]
http://mouse.owl#MA_0002755: ['', 'http://human.owl#NCI_C32951', '', 0.765652060508728]
http://mouse.owl#MA_0002754: ['', 'http://human.owl#NCI_C33714', '', 0.3939228355884552]
http://mouse.owl#MA_0002753: ['', 'http://human.owl#NCI_C12967', '', 0.6740885972976685]
http://mouse.owl#MA_0002752: ['', 'http://human.owl#NCI_C32977', '', 0.6873423457145691]
http://mouse.owl#MA_0002751: ['', 'http://human.owl#NCI_C33969', '', 0.7298470139503479]
http://mouse.owl#MA_0002750: ['', 'http://human.owl#NCI_C33588', '', 0.9309330582618713]
http://mouse.owl#MA_0002749: ['', 'http://human.owl#NCI_C12895', '', 0.7701386213302612]
http://mouse.owl#MA_0002747: ['', 'http://human.owl#NCI_C33848', '', 0.6927673816680908]
http://mouse.owl#MA_00

In [28]:
def transform_dict(original_dict):
    """
    Transforms a dictionary where URLs are the values into a dictionary where
    URLs are the keys and the values are concatenated labels associated with each URL.

    Args:
    original_dict (OrderedDict): The original dictionary with labels as keys and URLs as values.

    Returns:
    dict: A dictionary with URLs as keys and concatenated labels as values.
    """
    new_dict = {}
    for label, url in original_dict.items():
        if url in new_dict:
            new_dict[url] += " " + label  # Concatenating labels with space; change as needed
        else:
            new_dict[url] = label
    return new_dict

onto1_transformed_dict = transform_dict(onto1_dict)
onto2_transformed_dict = transform_dict(onto2_dict)

In [29]:
# Add labels
def add_labels(data, onto1_label, onto2_label):
    # Iterate through each key and update the list with labels
    for key, values in data.items():
        # First empty string in the list gets replaced by the label from onto1_label if available
        label1 = onto1_label.get(key)
        if label1 is not None:
            values[0] = label1

        # Second empty string in the list gets replaced by the label from onto2_label using the second entry's key if available
        label2_key = values[1]  # The second entry in the list is assumed to be a key for the onto2_label
        label2 = onto2_label.get(label2_key)
        if label2 is not None:
            values[2] = label2

    return data

matched_results_llm_with_labels = add_labels(matched_results_llm, onto1_transformed_dict, onto2_transformed_dict)

In [30]:
# Print the first entries from the dictionary
for onto1_class in list(matched_results_llm.keys())[:20]:
    print(f"{onto1_class}: {matched_results_llm[onto1_class]}")

http://mouse.owl#MA_0002758: ['levator auris longus', 'http://human.owl#NCI_C12315', 'fundus uteri', 0.4048202633857727]
http://mouse.owl#MA_0002757: ['periolivary nucleus', 'http://human.owl#NCI_C41416', 'perineural cell', 0.6656939387321472]
http://mouse.owl#MA_0002756: ['lateral spinal nucleus', 'http://human.owl#NCI_C12969', 'descending spinal cord tract', 0.7062833905220032]
http://mouse.owl#MA_0002755: ['lateral cervical nucleus', 'http://human.owl#NCI_C32951', 'lateral vestibular nucleus', 0.765652060508728]
http://mouse.owl#MA_0002754: ['neocortex', 'http://human.owl#NCI_C33714', 'sylvian cistern', 0.3939228355884552]
http://mouse.owl#MA_0002753: ['spinal cord intermediolateral nucleus', 'http://human.owl#NCI_C12967', 'ascending spinal cord tract', 0.6740885972976685]
http://mouse.owl#MA_0002752: ['nucleus approprius', 'http://human.owl#NCI_C32977', 'lenticular nucleus', 0.6873423457145691]
http://mouse.owl#MA_0002751: ['spinal cord ventral column', 'http://human.owl#NCI_C33969

## Combining & filtering

In [31]:
counter = 0
# Print the first entries from the dictionary
for onto1_class in list(matched_results_llm_with_labels.keys()):
    if matched_results_llm_with_labels[onto1_class][3] < 0.6:
        counter += 1
        print(f"{onto1_class}: {matched_results_llm_with_labels[onto1_class]}")
        
counter

http://mouse.owl#MA_0002758: ['levator auris longus', 'http://human.owl#NCI_C12315', 'fundus uteri', 0.4048202633857727]
http://mouse.owl#MA_0002754: ['neocortex', 'http://human.owl#NCI_C33714', 'sylvian cistern', 0.3939228355884552]
http://mouse.owl#MA_0002745: ['middle cerebellar peduncle', 'http://human.owl#NCI_C33114', 'middle cerebral artery branch', 0.5472418069839478]
http://mouse.owl#MA_0002741: ['brainstem white matter', 'http://human.owl#NCI_C32292', 'cerebral sulcus', 0.4498595893383026]
http://mouse.owl#MA_0002739: ['serous pericardium', 'http://human.owl#NCI_C33302', 'perineurium', 0.557588517665863]
http://mouse.owl#MA_0002738: ['fibrous pericardium', 'http://human.owl#NCI_C32601', 'fibrous coat of the liver', 0.45768895745277405]
http://mouse.owl#MA_0002728: ['larynx submucosa', 'http://human.owl#NCI_C12240', 'posterior wall of the oropharynx', 0.5467197299003601]
http://mouse.owl#MA_0002721: ['intercerebral commissure', 'http://human.owl#NCI_C32093', 'anterior infratent

694

In [32]:
# already included the perfect matches in the final_matching_results
final_matching_results = exact_matches.copy() # in this we will store the final matches

In [33]:
backup_string_matching_results = {} # this is just for easier testing and can be removed in the end
backup_matched_results_llm_with_labels = {}

# Check for overlapping classes
counter = 0
overlapping_results_keys = []
for class_name, values in string_matching_results.items():
    class_2 = values[1]
    if class_2:
        if matched_results_llm_with_labels[class_name][1] == class_2:
            backup_string_matching_results[class_name] = values
            backup_matched_results_llm_with_labels[class_name] = matched_results_llm_with_labels[class_name]
            higher_score = values[3]
            label_higher_score = values[2]
            if matched_results_llm_with_labels[class_name][3] > higher_score:
                higher_score = matched_results_llm_with_labels[class_name][3]
                label_higher_score = matched_results_llm_with_labels[class_name][2]
            final_matching_results[class_name] = [values[0], values[1], label_higher_score, higher_score]
            overlapping_results_keys.append(class_name)
            counter += 1
        
counter
print(len(final_matching_results))

1354


In [34]:
# Remove overlapping keys
for key in overlapping_results_keys:
    string_matching_results.pop(key, None)
    matched_results_llm_with_labels.pop(key, None)

In [35]:
# Check how many of the same matches for String and LLM have a score higher 0.7 => almost all of them
counter = 0
for class_name, values in final_matching_results.items():
    if values[3] > 0.7:
        counter += 1
        # print(final_matching_results[class_name])
        
print(len(final_matching_results))
print(counter)

1354
1294


In [36]:
# method to calculate the score for a given dict of matched classes
# this methods enables us to calculate the String matching score for the results of the LLM and vice versa
def calc_score_for_matched_classes(matched_classes, metric, dict_sim_scores_llm = {}):
    matches_with_score = {}
    for class_name, values in matched_classes.items():
        label1 = values[0]
        class_2 = values[1]
        label2 = values[2]
        if label2:
            matching_score = 0
            if metric == "llm":
                if class_name in dict_sim_scores_llm:
                    matching_score = dict_sim_scores_llm[class_name][class_2]
            else:
                matching_score = execute_string_matching(metric, label1, label2) # calculate string matching score
            matches_with_score[class_name] = [label1, class_2, label2, matching_score]
        else:
            matches_with_score[class_name] = [label1, class_2, "", 0]
    
    return matches_with_score

In [37]:
string_matches_for_llm = calc_score_for_matched_classes(matched_results_llm_with_labels, "Jaccard")
llm_matches_for_string = calc_score_for_matched_classes(string_matching_results, "llm", dict_similarity_scores_llm)

**In total we now have 4 lists**:
- one list calculated with the best 1-to-1 matches for String matching
- one list calculated with the best 1-to-1 matches for LLM matching
- one list calculated with the 1-to-1 matches of the LLM matching but with the String matching score
- one list calculated with the 1-to-1 matches of the String matching but with the LLM matching score

We **need 4 lists as otherwise there is no possibility to merge the results** of the two methods. The matched classes for the different methods (String and LLM) contain many different results: <br><br>For example:
- String matching: http://mouse.owl#MA_0001230 matched to http://human.owl#NCI_C32861
- LLM matching: http://mouse.owl#MA_0001230 matched to http://human.owl#NCI_C33192

So in order to merge them we need to calculate the scores of the resulting lists using the other method, thus resulting in 4 lists.

#### Final String matching lists

In [38]:
# Print the first entries from the dictionary
empty_counter = 0
for onto1_class in list(string_matching_results.keys()):
    if (string_matching_results[onto1_class][1] == ""):
        empty_counter += 1
    if onto1_class == "http://mouse.owl#MA_0001230":
        print(f"{onto1_class}: {string_matching_results[onto1_class]}")
    if string_matching_results[onto1_class][1] == "http://human.owl#NCI_C33192":
        print(f"{onto1_class}: {string_matching_results[onto1_class]}")

http://mouse.owl#MA_0002326: ['intercostales internus', 'http://human.owl#NCI_C33192', 'obturator internus muscle', 0.408248290463863]
http://mouse.owl#MA_0001230: ['extrinsic auricular muscle', 'http://human.owl#NCI_C33778', 'thyroepiglottic muscle', 0.408248290463863]


In [39]:
empty_counter

178

In [40]:
# Print the first entries from the dictionary
for onto1_class in list(string_matches_for_llm.keys()):
    if onto1_class == "http://mouse.owl#MA_0001230":
        print(f"{onto1_class}: {string_matches_for_llm[onto1_class]}")
    if onto1_class == "http://mouse.owl#MA_0002326":
        print(f"{onto1_class}: {string_matches_for_llm[onto1_class]}")

http://mouse.owl#MA_0002326: ['intercostales internus', 'http://human.owl#NCI_C12625', 'interneuron', 0.0]
http://mouse.owl#MA_0001230: ['extrinsic auricular muscle', 'http://human.owl#NCI_C33192', 'obturator internus muscle', 0.2]


In [41]:
# http://mouse.owl#MA_0001230 matched to http://human.owl#NCI_C33192 as it has highest score for LLM
# http://mouse.owl#MA_0002326 matched to 

#### Final LLM matching lists

In [42]:
# Print the first entries from the dictionary
for onto1_class in list(matched_results_llm_with_labels.keys()):
    if onto1_class == "http://mouse.owl#MA_0001230":
        print(f"{onto1_class}: {matched_results_llm_with_labels[onto1_class]}")
    if onto1_class == "http://mouse.owl#MA_0002326":
        print(f"{onto1_class}: {matched_results_llm_with_labels[onto1_class]}")

http://mouse.owl#MA_0002326: ['intercostales internus', 'http://human.owl#NCI_C12625', 'interneuron', 0.6117238402366638]
http://mouse.owl#MA_0001230: ['extrinsic auricular muscle', 'http://human.owl#NCI_C33192', 'obturator internus muscle', 0.6077027320861816]


In [43]:
# Print the first entries from the dictionary
for onto1_class in list(llm_matches_for_string.keys()):
    if onto1_class == "http://mouse.owl#MA_0001230":
        print(f"{onto1_class}: {llm_matches_for_string[onto1_class]}")
    if llm_matches_for_string[onto1_class][1] == "http://human.owl#NCI_C33192":
        print(f"{onto1_class}: {llm_matches_for_string[onto1_class]}")

http://mouse.owl#MA_0002326: ['intercostales internus', 'http://human.owl#NCI_C33192', 'obturator internus muscle', 0.5459696054458618]
http://mouse.owl#MA_0001230: ['extrinsic auricular muscle', 'http://human.owl#NCI_C33778', 'thyroepiglottic muscle', 0.46689164638519287]


In [44]:
# This code sorts the class names of ontology 1 by score in descending order
# This step is important before the combining step as it ensures that first the classes with the highest score
# matches get combined and therefore for these classes less conflict occur as they are handled in the beginning
onto1_class_names_by_score = {}

for onto1_class in list(string_matching_results.keys()):
    string_match_result = string_matching_results[onto1_class]
    string_match_result_for_llm = string_matches_for_llm[onto1_class]
    llm_match_result = matched_results_llm_with_labels[onto1_class]
    llm_match_result_for_string = llm_matches_for_string[onto1_class]
    
    entries = [string_match_result, string_match_result_for_llm, llm_match_result, llm_match_result_for_string]
    highest_score_entry = max(entries, key=lambda x: x[-1])
    onto1_class_names_by_score[onto1_class] = highest_score_entry

# Sort the classes based on the maximum score in descending order
onto1_class_names_by_score = sorted(list(onto1_class_names_by_score.items()), key=lambda x: x[1][3], reverse=True)

sorted_onto1_class_names_by_score = OrderedDict(onto1_class_names_by_score)

In [45]:
# In this code block the four lists are taken and the best matching of the four is picked.
# The code handles conflicts (class2 already used in an earlier match for a different class1)
# It is not perfect as if in the end there are just conflicts that can't be solve these will just be added as empty results
# => but as this are only a few entries and the complexity of handling those will be quite large we will stick with these results for now


# is called a greedy approach (for report)
remaining_results = {} # here the remaining matches will be stored and in the end merged with the final_results
already_used_class2 = []
counter = 0
conflicts = {}

for onto1_class in list(sorted_onto1_class_names_by_score.keys()):
    string_match_result = string_matching_results[onto1_class]
    string_match_result_for_llm = string_matches_for_llm[onto1_class]
    llm_match_result = matched_results_llm_with_labels[onto1_class]
    llm_match_result_for_string = llm_matches_for_string[onto1_class]
    
    results = [string_match_result, string_match_result_for_llm, llm_match_result, llm_match_result_for_string]
    empty_class2_counter = 0
    
    while True:
        if len(results) == 0:
            break
        highest_score_entry = max(results, key=lambda x: x[-1])
        class2_highest_entry = highest_score_entry[1]
        if class2_highest_entry in already_used_class2: # class2 is already matched to another class1
            results.remove(highest_score_entry)
        else:
            break
    # if class2_highest_entry == "":
    #     print("No match found")
    if len(results) == 0 or class2_highest_entry == "": # conflict occur if either all matches the class2 is already in use or no match was found (no match theoretically never happens as LLM always get some match)
        counter += 1
        conflicts[onto1_class] = results
        remaining_results[onto1_class] = ["", "" ,"" ,0]
    else:
        already_used_class2.append(class2_highest_entry)
        remaining_results[onto1_class] = highest_score_entry

In [46]:
df_remaining_results = pd.DataFrame.from_dict(remaining_results, orient="index")
df_remaining_results.reset_index(inplace=True)
df_remaining_results.columns = ['Class1', 'Class1_label', 'Class2', 'Class2_label', 'Score']

In [47]:
print(len(string_matching_results))
print(len(df_remaining_results))

1383
1383


In [48]:
print(len(df_remaining_results[df_remaining_results["Class2"] == ""])) # => only a few class1 have matches that does conflict with other entries
conflicts

10


{'http://mouse.owl#MA_0001689': [],
 'http://mouse.owl#MA_0002724': [],
 'http://mouse.owl#MA_0001556': [['mesoduodenum', '', '', 0],
  ['mesoduodenum', 'http://human.owl#NCI_C33103', 'mesentery', 0.0],
  ['mesoduodenum', '', '', 0]],
 'http://mouse.owl#MA_0001089': [],
 'http://mouse.owl#MA_0001320': [],
 'http://mouse.owl#MA_0000186': [],
 'http://mouse.owl#MA_0001005': [],
 'http://mouse.owl#MA_0001649': [],
 'http://mouse.owl#MA_0001040': [],
 'http://mouse.owl#MA_0000990': []}

In [49]:
# Group by 'Class2' column
grouped = df_remaining_results.groupby('Class2')

# Filter groups that have more than one entry
duplicate_class2 = grouped.filter(lambda x: len(x) > 1)

In [50]:
# check how often the LLM score is higher than the LLM score on the classes of the string matching
# Does not check if LLM has higher matches than string matching!
score_count_llm_org = 0
score_count_llm_for_string = 0
equal_count = 0

for onto1_class in list(matched_results_llm_with_labels.keys()):
    llm_score_org = matched_results_llm_with_labels[onto1_class][3]
    llm_score_for_string = llm_matches_for_string[onto1_class][3]
    if llm_score_org > llm_score_for_string:
        score_count_llm_org += 1
    elif llm_score_org < llm_score_for_string:
        score_count_llm_for_string += 1
    else:
        equal_count += 1 # will be never triggered as LLM has very fine matching (lots of decimals)
        
print(score_count_llm_org)
print(score_count_llm_for_string)

1097
286


In [51]:
# Combine the remaining results with the matches we already filtered out as they are already handled in the beginning
final_matching_results.update(remaining_results)

In [52]:
print(len(final_matching_results))
print(len(onto1_dict)) # already a TODO more earlier in the code to check why difference of 1

2737
1805


In [53]:
# filtering with threshold
final_results_over_threshold = {}
threshold = 0.8

for class_name, values in final_matching_results.items():
    label1, class2, label2, score = values
    if score > threshold:
        final_results_over_threshold[class_name] = values
print(len(final_results_over_threshold))

1385


### Check how many conflicts if threshold is already set earlier in code (good for the report for validation of our method)

In [54]:
# Feedback Prof
# TODO sort four list by score so you dont take a lower score even though there exist a higher score match => SOLVED
# Write about related work for ontology matching frameworks, string matching, llm matching and use papers as reference there (max 1-2 pages)
# Also write about linktransformer even though we didn't use it in the final version (so Til gets the props he derserves)

In [55]:
# Not important, just there to see how many conflicts we have if we apply the threshold (filtering step) directly when combining the lists
test_remaining_results = {} # here the remaining matches will be stored and in the end merged with the final_results
already_used_class2 = []
counter = 0
conflicts = {}

for onto1_class in list(sorted_onto1_class_names_by_score.keys()):
    string_match_result = string_matching_results[onto1_class]
    string_match_result_for_llm = string_matches_for_llm[onto1_class]
    llm_match_result = matched_results_llm_with_labels[onto1_class]
    llm_match_result_for_string = llm_matches_for_string[onto1_class]
    
    results = [string_match_result, string_match_result_for_llm, llm_match_result, llm_match_result_for_string]
    empty_class2_counter = 0
    
    while True:
        if len(results) == 0:
            break
        highest_score_entry = max(results, key=lambda x: x[-1])
        class2_highest_entry = highest_score_entry[1]
        if class2_highest_entry in already_used_class2: # class2 is already matched to another class1
            results.remove(highest_score_entry)
        else:
            break
    # if class2_highest_entry == "":
    #     print("No match found")
    if len(results) == 0 or class2_highest_entry == "": # conflict occur if either all matches the class2 is already in use or no match was found (no match theoretically never happens as LLM always get some match)
        counter += 1
        conflicts[onto1_class] = results
        # test_remaining_results[onto1_class] = ["", "" ,"" ,0]
    else:
        if highest_score_entry[3] > 0.8:
            already_used_class2.append(class2_highest_entry)
            test_remaining_results[onto1_class] = highest_score_entry
            
print(len(string_matching_results))
print(len(test_remaining_results)) # => only that many matches fulfill the requirements of the defined threshold
print(len(conflicts))
# => this shows us that no classes of ontology1 have a conflict if we set a threshold of 0.8
# => most of the conflicts that occur either way don't have a higher score than the threshold

1383
183
1


In [56]:
# check for http://mouse.owl#MA_0001424
# http://mouse.owl#MA_0001424 has label cervical vertebra 4
# currently matched with http://human.owl#NCI_C32245 => has label C7_Vertebra
# but in the reference.rdf matched with http://human.owl#NCI_C32242 => has label C4_Vertebra with score 1
# REPORT when running the matching again the http://mouse.owl#MA_0001424 gets matched to http://human.owl#NCI_C32431
# => which has a score of 0.16 => reason: because now the elements where iterated differently the before matched label got now matched with
# another class but with same score 0.25 => http://mouse.owl#MA_0001436: ['sacral vertebra 3', 'http://human.owl#NCI_C32245', 0.25]
print(jaccard_similarity("cervical vertebra 4", "C4_Vertebra"))
print(jaccard_similarity("thoracic vertebra 10", "C4_Vertebra"))

0.0
0.0


## 4. Write to output rdf

In [57]:
# turn results into triplets
final_matches_out = []

for match in final_results_over_threshold:
    class1 = match
    class2 = final_results_over_threshold[match][1]
    score = final_results_over_threshold[match][3]
    final_matches_out.append((class1, class2, score, "="))

final_matches_out

[('http://mouse.owl#MA_0000003', 'http://human.owl#NCI_C12919', 1, '='),
 ('http://mouse.owl#MA_0000004', 'http://human.owl#NCI_C33816', 1, '='),
 ('http://mouse.owl#MA_0000007', 'http://human.owl#NCI_C12429', 1, '='),
 ('http://mouse.owl#MA_0000009', 'http://human.owl#NCI_C12472', 1, '='),
 ('http://mouse.owl#MA_0000010', 'http://human.owl#NCI_C12686', 1, '='),
 ('http://mouse.owl#MA_0000011', 'http://human.owl#NCI_C12374', 1, '='),
 ('http://mouse.owl#MA_0000012', 'http://human.owl#NCI_C12705', 1, '='),
 ('http://mouse.owl#MA_0000015', 'http://human.owl#NCI_C13056', 1, '='),
 ('http://mouse.owl#MA_0000016', 'http://human.owl#NCI_C12755', 1, '='),
 ('http://mouse.owl#MA_0000018', 'http://human.owl#NCI_C12788', 1, '='),
 ('http://mouse.owl#MA_0000020', 'http://human.owl#NCI_C13062', 1, '='),
 ('http://mouse.owl#MA_0000022', 'http://human.owl#NCI_C12799', 1, '='),
 ('http://mouse.owl#MA_0000023', 'http://human.owl#NCI_C12419', 1, '='),
 ('http://mouse.owl#MA_0000024', 'http://human.owl#

In [58]:
# Initialize graph
g = rdflib.Graph()

# Define namespaces
KNOWLEDGEWEB = rdflib.Namespace("http://knowledgeweb.semanticweb.org/heterogeneity/alignment#")
g.bind("kw", KNOWLEDGEWEB)

# Create the root element for the alignment
alignment = rdflib.URIRef("http://example.org/alignment")

# Add basic alignment properties
g.add((alignment, rdflib.namespace.RDF.type, KNOWLEDGEWEB.Alignment))
g.add((alignment, KNOWLEDGEWEB.xml, rdflib.Literal("yes")))
g.add((alignment, KNOWLEDGEWEB.level, rdflib.Literal("0")))
g.add((alignment, KNOWLEDGEWEB.type, rdflib.Literal("??")))

# Add each match to the graph
for entity1, entity2, measure, relation in final_matches_out:
    cell = rdflib.URIRef(f"http://example.org/cell/{entity1.split('#')[-1]}_{entity2.split('#')[-1]}")
    g.add((cell, rdflib.namespace.RDF.type, KNOWLEDGEWEB.Cell))
    g.add((cell, KNOWLEDGEWEB.entity1, rdflib.URIRef(entity1)))
    g.add((cell, KNOWLEDGEWEB.entity2, rdflib.URIRef(entity2)))
    g.add((cell, KNOWLEDGEWEB.measure, rdflib.Literal(measure, datatype=rdflib.namespace.XSD.float)))
    g.add((cell, KNOWLEDGEWEB.relation, rdflib.Literal(relation)))
    g.add((alignment, KNOWLEDGEWEB.map, cell))

# Serialize the graph to an RDF file (e.g., in RDF/XML format)
with open("ontology_matching_results.rdf", "wb") as f:
    f.write(g.serialize(format='pretty-xml').encode("utf-8"))

# Optionally, print out the graph in RDF/XML format for inspection
print(g.serialize(format='pretty-xml'))


<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
  xmlns:kw="http://knowledgeweb.semanticweb.org/heterogeneity/alignment#"
>
  <kw:Alignment rdf:about="http://example.org/alignment">
    <kw:xml>yes</kw:xml>
    <kw:level>0</kw:level>
    <kw:type>??</kw:type>
    <kw:map>
      <kw:Cell rdf:about="http://example.org/cell/MA_0000003_NCI_C12919">
        <kw:entity1 rdf:resource="http://mouse.owl#MA_0000003"/>
        <kw:entity2 rdf:resource="http://human.owl#NCI_C12919"/>
        <kw:measure rdf:datatype="http://www.w3.org/2001/XMLSchema#float">1</kw:measure>
        <kw:relation>=</kw:relation>
      </kw:Cell>
    </kw:map>
    <kw:map>
      <kw:Cell rdf:about="http://example.org/cell/MA_0000004_NCI_C33816">
        <kw:entity1 rdf:resource="http://mouse.owl#MA_0000004"/>
        <kw:entity2 rdf:resource="http://human.owl#NCI_C33816"/>
        <kw:measure rdf:datatype="http://www.w3.org/2001/XMLSchema#float">1</kw:measure>
  

Output format example:
```
<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF xmlns="http://knowledgeweb.semanticweb.org/heterogeneity/alignment"
	 xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
	 xmlns:xsd="http://www.w3.org/2001/XMLSchema#">

<Alignment>
<xml>yes</xml>
<level>0</level>
<type>??</type>

<map>
	<Cell>
		<entity1 rdf:resource="http://mouse.owl#MA_0002401"/>
		<entity2 rdf:resource="http://human.owl#NCI_C52561"/>
		<measure rdf:datatype="xsd:float">1.0</measure>
		<relation>=</relation>
	</Cell>
</map>
```